<a href="https://colab.research.google.com/github/meisisoiisme/FH-ML_PyTorch/blob/main/Untitled11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imoprts

1.   ML Imports
2.   Creating Dataset
3.   Handeling Dataset info

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset

import numpy as np
import random
from PIL import Image, ImageDraw
import os

import csv
import cv2

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

Generate Images for Training your Network

    # Example usage: Copy Code, remove lines containing information, adapt filepath if nessesary

1.   ImageGenerator

    output_directory = "./training_images"
    image_gen = ImageGenerator(output_directory, num_classes=5, images_per_class=100, noise_factor=20)
    image_gen.generate_images()

2.   LandmarksImageGenerator

    output_directory_landmarks = "./training_images_with_landmarks"
    landmarks_gen = LandmarksImageGenerator(output_directory_landmarks, num_classes=5, images_per_class=100, noise_factor=20, num_landmarks=5)
    landmarks_gen.generate_images_with_landmarks()

    # Parameters
    num_samples = 500
    num_classes = 5
    image_size = 64
    batch_size = 32
    num_epochs = 10
    data_path = "./training_images"
    landmarks_csv_path = "./training_images_with_landmarks/landmarks.csv"

3.   CustomRandomDataset

    # Create the custom dataset
    custom_dataset = CustomRandomDataset(num_samples, num_classes, image_size, data_path, landmarks_csv_path)





In [29]:
class ImageGenerator:
    def __init__(self, output_dir, num_classes=5, images_per_class=100, image_size=(224, 224), noise_factor=20):
        self.output_dir = output_dir
        self.num_classes = num_classes
        self.images_per_class = images_per_class
        self.image_size = image_size
        self.noise_factor = noise_factor

    def generate_images(self):
        os.makedirs(self.output_dir, exist_ok=True)

        for class_idx in range(self.num_classes):
            class_dir = os.path.join(self.output_dir, f"class_{class_idx}")
            os.makedirs(class_dir, exist_ok=True)

            for i in range(self.images_per_class):
                base_image = np.random.randint(0, 256, size=(self.image_size[1], self.image_size[0], 3), dtype=np.uint8)
                noisy_image = base_image + np.random.randint(-self.noise_factor, self.noise_factor + 1, size=base_image.shape)
                noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

                pil_image = Image.fromarray(noisy_image)
                image_path = os.path.join(class_dir, f"image_{i}.png")
                pil_image.save(image_path)

        print(f"{self.num_classes} classes with {self.images_per_class} images each generated and saved in {self.output_dir}")

class LandmarksImageGenerator:
    def __init__(self, output_dir, num_classes=5, images_per_class=100, image_size=(224, 224), noise_factor=20, num_landmarks=5):
        self.output_dir = output_dir
        self.num_classes = num_classes
        self.images_per_class = images_per_class
        self.image_size = image_size
        self.noise_factor = noise_factor
        self.num_landmarks = num_landmarks

    def generate_images_with_landmarks(self):
        os.makedirs(self.output_dir, exist_ok=True)

        csv_file_path = os.path.join(self.output_dir, "landmarks.csv")
        with open(csv_file_path, mode='w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['Image Path', 'Class', 'Landmark 1 X', 'Landmark 1 Y', 'Landmark 2 X', 'Landmark 2 Y', '...'])

            for class_idx in range(self.num_classes):
                class_dir = os.path.join(self.output_dir, f"class_{class_idx}")
                os.makedirs(class_dir, exist_ok=True)

                for i in range(self.images_per_class):
                    base_image = np.random.randint(0, 256, size=(self.image_size[1], self.image_size[0], 3), dtype=np.uint8)
                    noisy_image = base_image + np.random.randint(-self.noise_factor, self.noise_factor + 1, size=base_image.shape)
                    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

                    landmarks = []
                    for landmark_idx in range(self.num_landmarks):
                        landmark_x = random.randint(0, self.image_size[0])
                        landmark_y = random.randint(0, self.image_size[1])
                        landmarks.extend([landmark_x, landmark_y])

                    pil_image = Image.fromarray(noisy_image)
                    image_path = os.path.join(class_dir, f"image_{i}.png")
                    pil_image.save(image_path)

                    csv_writer.writerow([image_path, class_idx] + landmarks)

        print(f"{self.num_classes} classes with {self.images_per_class} images each generated and saved in {self.output_dir}")
        print(f"Landmark information saved in {csv_file_path}")

class CustomRandomDataset(Dataset):
    def __init__(self, num_samples, num_classes, image_size, data_path=None, landmarks_csv_path=None):
        self.num_samples = num_samples
        self.num_classes = num_classes
        self.image_size = image_size
        self.data = []
        self.labels = []
        self.data_path = data_path
        self.landmarks_csv_path = landmarks_csv_path
        self.num_landmarks = 5  # Assuming the default number of landmarks

        self.load_data()

    def load_data(self):
        if self.data_path is not None:
            for label in range(self.num_classes):
                label_path = os.path.join(self.data_path, f"{label}")
                if os.path.exists(label_path):
                    class_images = [os.path.join(label_path, image_file) for image_file in os.listdir(label_path)]
                    if len(class_images) == 0:
                        print(f"Warning: Class {label} has no images.")
                        continue

                    # Ensure the number of samples does not exceed the available images
                    num_samples_for_class = min(self.num_samples // self.num_classes, len(class_images))
                    sampled_images = random.sample(class_images, num_samples_for_class)

                    self.data.extend(sampled_images)
                    self.labels.extend([label] * num_samples_for_class)

    def load_image(self, image_path):
        image = Image.open(image_path).convert("RGB")
        image = image.resize((self.image_size, self.image_size))
        return transforms.ToTensor()(image)

    def load_landmarks(self, csv_path):
        landmarks = np.zeros((len(self.data), 2 * self.num_landmarks), dtype=np.float32)

        with open(csv_path, mode='r') as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader)  # Skip header row

            for i, row in enumerate(csv_reader):
                image_path, _, *landmark_values = row
                image_index = self.data.index(image_path)
                landmarks[image_index] = list(map(float, landmark_values))

        return torch.tensor(landmarks)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_path = self.data[index]
        label = self.labels[index]
        image = self.load_image(image_path)
        landmarks = self.load_landmarks(self.landmarks_csv_path)
        return image, label, landmarks


DEFINE CNN MODELS



1.   CNN - Custom
2.   AlexNet
3.   LenNet5
4.   ResNet - Pretrained Model: ResNet50

In [36]:
class CNN(nn.Module):
    def __init__(self, num_classes, image_size):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 * (image_size // 4) * (image_size // 4), 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

class AlexNet(nn.Module):
    def __init__(self, num_classes=6):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

class LeNet5(nn.Module):
    def __init__(self, num_classes=6):
        super(LeNet5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 120, kernel_size=5),
            nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.Linear(120 * 53 * 53, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

class ResNetModel(nn.Module):
    def __init__(self, num_classes=6):
        super(ResNetModel, self).__init__()
        self.model = models.resnet50(pretrained=True)
        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.model(x)



# Assuming you have already created the custom dataset and data loaders
train_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Example usage:
model_trainer = ModelTrainer(model, train_loader, val_loader, test_loader, criterion, optimizer, landmarks_csv_path=landmarks_csv_path, num_epochs=10)
model_trainer.train()
model_trainer.evaluate_and_save_results()

In [17]:
class ModelTrainer:
    def __init__(self, model, train_loader, val_loader, test_loader, criterion, optimizer, landmarks_csv_path=None, num_epochs=30):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.landmarks_csv_path = landmarks_csv_path
        self.num_epochs = num_epochs

    @property
    def device(self):
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def train(self):
        self.model.to(self.device)

        for epoch in range(self.num_epochs):
            self.model.train()
            for inputs, labels, landmarks in self.train_loader:
                inputs, labels, landmarks = inputs.to(self.device), labels.to(self.device), landmarks.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

            self.model.eval()
            val_loss = 0.0
            correct = 0
            total = 0

            with torch.no_grad():
                for inputs, labels, landmarks in self.val_loader:
                    inputs, labels, landmarks = inputs.to(self.device), labels.to(self.device), landmarks.to(self.device)

                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, labels)
                    val_loss += loss.item()

                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            print(f'Epoch {epoch+1}/{self.num_epochs}, Loss: {val_loss/len(self.val_loader)}, Validation Accuracy: {correct/total}')

    def grad_cam(self, input_tensor, target_class=None):
        self.model.eval()
        device = next(self.model.parameters()).device

        # Get the output from the final convolutional layer
        final_conv_layer = None
        for layer in self.model.children():
            if isinstance(layer, nn.Conv2d):
                final_conv_layer = layer

        # Get the gradients with respect to the output of the final convolutional layer
        self.model.zero_grad()
        output = self.model(input_tensor.to(device))
        if target_class is None:
            target_class = torch.argmax(output)
        output[:, target_class].backward()

        # Get the feature map from the final convolutional layer
        feature_maps = final_conv_layer.weight.grad
        alpha = torch.mean(feature_maps, dim=(2, 3), keepdim=True)

        # Perform weighted combination to get the heatmap
        heatmap = torch.sum(alpha * feature_maps, dim=1, keepdim=True)
        heatmap = nn.functional.relu(heatmap)

        # Normalize the heatmap
        heatmap /= torch.max(heatmap)

        return heatmap

    def overlay_heatmap(self, image, heatmap):
        heatmap = heatmap.squeeze().cpu().numpy()
        heatmap = np.uint8(255 * heatmap)
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        overlayed_image = cv2.addWeighted(image, 0.5, heatmap, 0.5, 0)
        return overlayed_image

    def evaluate_and_save_results(self):
        self.model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels, landmarks in self.test_loader:
                inputs, labels, landmarks = inputs.to(self.device), labels.to(self.device), landmarks.to(self.device)
                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculate confusion matrix
        cm = confusion_matrix(all_labels, all_preds)

        # Plot confusion matrix
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.colorbar()
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.show()

In [30]:
output_directory = "./training_images"
image_gen = ImageGenerator(output_directory, num_classes=5, images_per_class=100, noise_factor=20)
image_gen.generate_images()

output_directory_landmarks = "./training_images_with_landmarks"
landmarks_gen = LandmarksImageGenerator(output_directory_landmarks, num_classes=5, images_per_class=100, noise_factor=20, num_landmarks=5)
landmarks_gen.generate_images_with_landmarks()

5 classes with 100 images each generated and saved in ./training_images
5 classes with 100 images each generated and saved in ./training_images_with_landmarks
Landmark information saved in ./training_images_with_landmarks/landmarks.csv


In [35]:
# Parameters
#num_samples = 100  # Make sure this is a positive value
#num_classes = 5
image_size = 64
batch_size = 32
num_epochs = 10
data_path = "./training_images"
landmarks_csv_path = "./training_images_with_landmarks/landmarks.csv"

# Create the custom dataset
custom_dataset = CustomRandomDataset(100, 5, image_size, data_path, landmarks_csv_path)
custom_val_dataset = CustomRandomDataset(num_samples // 8, num_classes, image_size, data_path, landmarks_csv_path)
custom_test_dataset = CustomRandomDataset(num_samples // 5, num_classes, image_size, data_path, landmarks_csv_path)

# Assuming you have already created the custom dataset and data loaders
train_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(custom_val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(custom_test_dataset, batch_size=batch_size, shuffle=False)



ValueError: num_samples should be a positive integer value, but got num_samples=0

In [27]:
model_trainer = ModelTrainer(model, train_loader, val_loader, test_loader, criterion, optimizer, landmarks_csv_path=landmarks_csv_path, num_epochs=10)
model_trainer.train()
model_trainer.evaluate_and_save_results()

IndexError: list index out of range

In [13]:
# Parameters
num_samples = 1000
num_classes = 6
image_size = 64
batch_size = 32
num_epochs = 30

# Create the custom dataset
custom_dataset = CustomRandomDataset(num_samples, num_classes, image_size)

# Create data loaders
train_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Initialize the CNN model
model = CNN(num_classes, image_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Use landmarks for images
landmarks = {
    0: "Landmark for class 0",
    1: "Landmark for class 1",
    2: "Landmark for class 2",
    3: "Landmark for class 3",
    4: "Landmark for class 4",
    5: "Landmark for class 5"
}

for label, landmark in landmarks.items():
    print(f"Landmark for class {label}: {landmark}")

Epoch 1/30, Loss: 1.796120285987854
Epoch 2/30, Loss: 1.793994426727295
Epoch 3/30, Loss: 1.7761523723602295
Epoch 4/30, Loss: 1.8070263862609863
Epoch 5/30, Loss: 1.7776235342025757
Epoch 6/30, Loss: 1.7696566581726074
Epoch 7/30, Loss: 1.822173833847046
Epoch 8/30, Loss: 1.752305507659912
Epoch 9/30, Loss: 1.5194302797317505
Epoch 10/30, Loss: 1.474936842918396
Epoch 11/30, Loss: 1.5054572820663452
Epoch 12/30, Loss: 0.833915650844574
Epoch 13/30, Loss: 0.7960186004638672
Epoch 14/30, Loss: 0.26359787583351135
Epoch 15/30, Loss: 0.3175039291381836
Epoch 16/30, Loss: 0.2553141713142395
Epoch 17/30, Loss: 0.10295675694942474
Epoch 18/30, Loss: 0.03644685819745064
Epoch 19/30, Loss: 0.08832476288080215
Epoch 20/30, Loss: 0.031078940257430077
Epoch 21/30, Loss: 0.022354021668434143
Epoch 22/30, Loss: 0.01790250465273857
Epoch 23/30, Loss: 0.01458773110061884
Epoch 24/30, Loss: 0.014260511845350266
Epoch 25/30, Loss: 0.009188798256218433
Epoch 26/30, Loss: 0.009971672669053078
Epoch 27/30